In [1]:
import sys
sys.path.append('tm_navigator')

In [2]:
import re
from collections import Counter
import numpy as np
import scipy.sparse as sp
from scipy.spatial.distance import pdist, squareform
import codecs
import h5py
import itertools as it

import sqlalchemy as sa
import sqlalchemy.ext.hybrid
import sqlalchemy.ext.declarative as sa_dec
import sqlalchemy_searchable as searchable
from sqlalchemy_sqlschema import maintain_schema

from tm_navigator.models import *

In [3]:
engine = sa.create_engine('postgresql+psycopg2://@localhost/tm_navigator_dev', echo=0)
sa.orm.configure_mappers()

In [4]:
Session = sa.orm.sessionmaker(bind=engine)

from contextlib import contextmanager

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [5]:
Base.metadata.create_all(engine,
                         tables=map(lambda c: c.__table__, models_public))

In [6]:
with session_scope() as session:
    ds = DatasetMeta()
    session.add(ds)
    ds = session.query(DatasetMeta).order_by(DatasetMeta.id.desc()).first()
    print(ds)
    
    ds.activate_schemas()
    Base.metadata.create_all(engine,
                             tables=map(lambda c: c.__table__, models_dataset))

<DatasetMeta(id=1, schema_name='tmnav_dataset_1', title=None)>


In [7]:
with session_scope() as session:
    ds = session.query(DatasetMeta).order_by(DatasetMeta.id.desc()).first()
    session.add(TopicModelMeta(dataset=ds))
    tm = session.query(TopicModelMeta).order_by(TopicModelMeta.id.desc()).first()
    print(tm)
    
    tm.activate_schemas()
    Base.metadata.create_all(engine,
                             tables=map(lambda c: c.__table__, models_topic + models_assessment))

<TopicModelMeta(dataset_id=1, id=1, schema_name='tmnav_topicmodel_1', title=None)>


In [12]:
with session_scope() as session:
    session.query(TopicModelMeta).delete()
    session.query(DatasetMeta).delete()

In [13]:
with session_scope() as session:
    schemas_matching = session.query('schema_name from information_schema.schemata') \
        .filter(sa.column('schema_name').startswith('tmnav_')) \
        .subquery()
    schemas_used = session.query(DatasetMeta.schema_name) \
        .union(session.query(TopicModelMeta.schema_name))
#     schemas_unused = session.query('schema_name').select_from(schemas_matching) \
#         .filter(sa.column('schema_name').notin_(schemas_used))
    schemas_unused = session.query('schema_name').select_from(schemas_matching)
    for schema, in schemas_unused:
        session.execute(sa.schema.DropSchema(name=schema, cascade=True))

In [14]:
with session_scope() as session:
    SchemaMixin.activate_public_schema(session)
    Base.metadata.drop_all(engine)

In [5]:
from sqlalchemy_utils import aggregates

aggregates.local_condition = lambda prop, objects: sa.literal(True)

class ListSession(list):
    def execute(self, query):
        print(query)
        return session.execute(query)

def update_aggregates(*classes_modified):
    aggregates.manager.construct_aggregate_queries(
        ListSession([c() for c in classes_modified]),
        None)

In [27]:
import csv
from io import StringIO
from contextlib import closing

def to_csv(rows, csv_f):
    firstrow = next(rows)
    fieldnames = firstrow.keys()

    writer = csv.DictWriter(csv_f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow(firstrow)
    writer.writerows(rows)

In [6]:
def copy_to_table(session, model, rows):
    firstrow = next(rows)
    fieldnames = firstrow.keys()

    with closing(StringIO()) as csv_f:
        writer = csv.DictWriter(csv_f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(firstrow)
        writer.writerows(rows)
        csv_f.seek(0)

        with session.connection().connection.cursor() as cursor:
            cursor.copy_expert('copy %s (%s) from stdin with csv header' % (model.__tablename__, ', '.join(fieldnames)), csv_f)
            
    
    update_aggregates(model)

In [7]:
with session_scope() as session:
    ds = session.query(DatasetMeta).filter_by(id=1).one()    
    ds.activate_schemas()

In [35]:
with open('data/modalities.csv', 'w') as csv_f:
    to_csv(({'id': i+1, 'name': n} for i, n in enumerate(['words', 'authors'])), csv_f)

In [33]:
with h5py.File('data/data.hdf') as h5f:
    metadata = h5f['metadata'][...]

In [47]:
with open('data/dictionary.mmro.txt') as f, open('data/terms.csv', 'w') as csv_f:
    rows = (dict(id=i, modality_id=1, text=line.strip())
            for i, line in enumerate(f))
    
    doc_authors = [(i, author.strip())
         for i, m in enumerate(metadata)
         for author in m['authors'].split(',')
    ]
    authors = {a for d, a in doc_authors}
    authors_terms = {a: dict(id=i, modality_id=2, text=a)
                     for i, a in enumerate(authors)}
    rows = it.chain(rows, authors_terms.values())
    
    to_csv(rows, csv_f)

In [38]:
with open('data/documents.csv', 'w') as csv_f:
    rows = (
        dict(id=i, title=t['title'], file_name=t['filename'], slug=t['slug'],
             source=re.sub(r'^\d{4}-([A-Z]+)(\d+)/.+', r'\1-\2', t['filename']),
             html=open('data/html_sprites/%s.html' % t['filename']).read())
        for i, t in enumerate(metadata)
    )
    to_csv(rows, csv_f)

In [48]:
with open('data/document_terms.csv', 'w') as csv_f, open('data/documents.mmro.txt') as f:
    rows = (dict(document_id=d, modality_id=1, term_id=w, count=cnt)
             for d, line in enumerate(f)
             for w, cnt in Counter(int(dw.split()[0]) for dw in line.split(';')[:-1]).items())

    rows = it.chain(rows,
                    (dict(document_id=d, modality_id=2, term_id=authors_terms[a]['id'], count=1)
                     for d, a in doc_authors))
    to_csv(rows, csv_f)

In [52]:
phi = sp.coo_matrix(np.load('data/phi.npy'))
theta = sp.coo_matrix(np.load('data/theta.npy'))

pwt = phi.A
ptd = theta.A

pd = 1.0 / theta.shape[1]
pt = (ptd * pd).sum(1)
pw = (pwt * pt).sum(1)
ptw = pwt * pt / pw[:, np.newaxis]
pdt = ptd * pd / pt[:, np.newaxis]

t_to_id = lambda level, t: level * 1000 + t

In [51]:
with open('data/topics.csv', 'w') as csv_f:    
    rows = (dict(id=t_to_id(1, t), type='foreground' if t < 50 else 'background', probability=p)
            for t, p in enumerate(pt))
    rows = it.chain(rows, (dict(id=0, type='foreground', probability=1),))
    to_csv(rows, csv_f)

In [53]:
with open('data/topic_terms.csv', 'w') as csv_f:
    rows = (dict(topic_id=t_to_id(1, t), modality_id=1, term_id=w, prob_wt=val, prob_tw=ptw[w, t])
            for w, t, val in zip(phi.row, phi.col, phi.data))
    to_csv(rows, csv_f)

In [54]:
with open('data/document_topics.csv', 'w') as csv_f:
    rows = (dict(topic_id=t_to_id(1, t), document_id=d, prob_td=val, prob_dt = pdt[t, d])
            for t, d, val in zip(theta.row, theta.col, theta.data))
    to_csv(rows, csv_f)

In [57]:
with open('data/topic_edges.csv', 'w') as csv_f:
    rows = (dict(parent_id=0, child_id=t_to_id(1, t), probability=p)
            for t, p in enumerate(pt))
    to_csv(rows, csv_f)

In [8]:
with session_scope() as session:
    session.add(Modality(name='words'))

In [9]:
with open('data/dictionary.mmro.txt') as f, session_scope() as session:
    modality = session.query(Modality).filter(Modality.name == 'words').one()
    rows = (dict(id=i, modality_id=modality.id, text=line.strip())
            for i, line in enumerate(f))
    copy_to_table(session, Term, rows)
    
    print(session.query(Term).count())

UPDATE modalities SET count=(SELECT count(*) AS count_1 
FROM terms 
WHERE modalities.id = terms.modality_id) WHERE :param_1
7805


In [10]:
with h5py.File('data/data.hdf') as h5f:
    metadata = h5f['metadata'][...]

In [11]:
%%time
with session_scope() as session:
    rows = (
        dict(id=i, title=t['title'], file_name=t['filename'], slug=t['slug'],
             source=re.sub(r'^\d{4}-([A-Z]+)(\d+)/.+', r'\1-\2', t['filename']),
             html=open('data/html_sprites/%s.html' % t['filename']).read())
        for i, t in enumerate(metadata)
    )
    copy_to_table(session, Document, rows)
    
    print(session.query(Document).count())

1061
CPU times: user 1.14 s, sys: 80 ms, total: 1.22 s
Wall time: 2.37 s


In [12]:
with session_scope() as session:
    session.add(Modality(name='authors'))

In [13]:
%%time
with session_scope() as session:
    m = session.query(Modality).filter(Modality.name == 'authors').one()
    
    doc_authors = [(i, author.strip())
         for i, m in enumerate(metadata)
         for author in m['authors'].split(',')
    ]
    authors = {a for d, a in doc_authors}
    authors_terms = {a: Term(modality=m, text=a)
                     for a in authors}
    session.add_all(
        DocumentTerm(document_id=d, term=authors_terms[a], count=1)
        for d, a in doc_authors)

CPU times: user 1.76 s, sys: 104 ms, total: 1.86 s
Wall time: 3.03 s


In [14]:
%%time
with open('data/documents.mmro.txt') as f, session_scope() as session:
    m_id = session.query(Modality.id).filter(Modality.name == 'words').scalar()
    
    rows = (dict(document_id=d, modality_id=m_id, term_id=w, count=cnt)
             for d, line in enumerate(f)
             for w, cnt in Counter(int(dw.split()[0]) for dw in line.split(';')[:-1]).items())

    copy_to_table(session, DocumentTerm, rows)
    
    print(session.query(DocumentTerm.modality_id, sa.func.count()).group_by(DocumentTerm.modality_id).all())

UPDATE terms SET count=(SELECT coalesce(sum(document_terms.count), :param_1) AS coalesce_1 
FROM document_terms 
WHERE terms.modality_id = document_terms.modality_id AND terms.id = document_terms.term_id) WHERE :param_2
[(1, 314081), (2, 2315)]
CPU times: user 2.82 s, sys: 44 ms, total: 2.87 s
Wall time: 18.5 s


In [15]:
%%time
with open('data/documents.mmro.txt') as f, session_scope() as session:
    m_id = session.query(Modality.id).filter(Modality.name == 'words').scalar()

    id_cnt = it.count()
    rows = (dict(id=next(id_cnt), document_id=d, modality_id=m_id, term_id=w, start_pos=s, end_pos=e)
            for d, line in enumerate(f)
            for w, s, e in (map(int, dw.split()) for dw in line.split(';')[:-1]))

    copy_to_table(session, DocumentContent, rows)

CPU times: user 9.29 s, sys: 164 ms, total: 9.46 s
Wall time: 46 s


In [17]:
phi = sp.coo_matrix(np.load('data/phi.npy'))
theta = sp.coo_matrix(np.load('data/theta.npy'))

In [18]:
pwt = phi.A
ptd = theta.A

pd = 1.0 / theta.shape[1]
pt = (ptd * pd).sum(1)
pw = (pwt * pt).sum(1)
ptw = pwt * pt / pw[:, np.newaxis]
pdt = ptd * pd / pt[:, np.newaxis]

In [19]:
with session_scope() as session:
    ds = session.query(TopicModelMeta).filter_by(id=1).one()    
    ds.activate_schemas()

In [20]:
%%time
with session_scope() as session:
    m_id = session.query(Modality.id).filter(Modality.name == 'words').scalar()
    
    session.add(Topic(id=0, type='foreground'))

    t_to_id = lambda level, t: level * 1000 + t

    copy_to_table(session, Topic,
                  (dict(id=t_to_id(1, t), type='foreground' if t < 50 else 'background')
                   for t in range(phi.shape[1])
                  ))
    rows = (dict(topic_id=t_to_id(1, t), modality_id=m_id, term_id=w, prob_wt=val, prob_tw=ptw[w, t])
            for w, t, val in zip(phi.row, phi.col, phi.data))
    copy_to_table(session, TopicTerm, rows)

    rows = (dict(topic_id=t_to_id(1, t), document_id=d, prob_td=val, prob_dt = pdt[t, d])
            for t, d, val in zip(theta.row, theta.col, theta.data))
    copy_to_table(session, DocumentTopic, rows)
            
    
    print(session.query(Topic.level, sa.func.count()).group_by(Topic.level).all())
    print(session.query(Topic.level, sa.func.count()).join(DocumentTopic).group_by(Topic.level).all())
    print(session.query(Topic.level, sa.func.count()).join(TopicTerm).group_by(Topic.level).all())

UPDATE topics SET probability=(SELECT coalesce(sum(document_topics.prob_td), :param_1) AS coalesce_1 
FROM document_topics 
WHERE topics.id = document_topics.topic_id) WHERE :param_2
[(0, 1), (1, 60)]
[(1, 14161)]
[(1, 120936)]
CPU times: user 3.64 s, sys: 8 ms, total: 3.65 s
Wall time: 9.33 s


In [21]:
with session_scope() as session:
    root = session.query(Topic).filter(Topic.level == 0).one()
    q = session.query(Topic).filter(Topic.level == 1)
    session.add_all(TopicEdge(parent=root, child=t, probability=t.probability) for t in q)

In [22]:
%%time
with open('data/ptdw.txt') as fp, session_scope() as session:
    id_cnt = it.count()
    rows = (dict(document_content_id=next(id_cnt), topic_id=1000 + int(t))
            for d, linep in enumerate(fp)
            for t in linep.split())

    copy_to_table(session, DocumentContentTopic, rows)

CPU times: user 4.8 s, sys: 56 ms, total: 4.86 s
Wall time: 29 s


In [23]:
%%time
with session_scope() as session:
    distances = squareform(pdist(theta.A.T, 'cosine'))

    rows = (dict(a_id=i, b_id=sim_i, similarity=1 - row[sim_i])
            for i, row in enumerate(distances)
            for sim_i in row.argsort()[:31]
            if sim_i != i)
    copy_to_table(session, DocumentSimilarity, rows)

CPU times: user 556 ms, sys: 0 ns, total: 556 ms
Wall time: 1.42 s


In [24]:
%%time
with session_scope() as session:
    distances = squareform(pdist(phi.A.T, 'cosine'))

    rows = (dict(a_id=t_to_id(1, i), b_id=t_to_id(1, sim_i), similarity=1 - row[sim_i])
            for i, row in enumerate(distances)
            for sim_i in row.argsort()[:]
            if sim_i != i)
    copy_to_table(session, TopicSimilarity, rows)

CPU times: user 72 ms, sys: 0 ns, total: 72 ms
Wall time: 175 ms


In [25]:
%%time
with session_scope() as session:
    m_id = session.query(Modality.id).filter(Modality.name == 'words').scalar()
    distances = squareform(pdist(phi.A, 'cosine'))

    rows = (dict(a_modality_id=m_id, b_modality_id=m_id, a_id=i, b_id=sim_i, similarity=1 - row[sim_i])
            for i, row in enumerate(distances)
            for sim_i in row.argsort()[:20]
            if sim_i != i)
    copy_to_table(session, TermSimilarity, rows)

CPU times: user 8.76 s, sys: 224 ms, total: 8.98 s
Wall time: 13.7 s


In [26]:
with session_scope() as session:
    session.execute('''
    with probs as (
        select
            topic_id,
            sum(prob_td / (select count(*) from documents)) as probability
        from document_topics
        group by topic_id
    )
    
    update topics
    set probability = probs.probability
    from probs
    where topics.id = probs.topic_id
    ''')
    print(session.query(Topic.level, sa.func.sum(Topic.probability)).group_by(Topic.level).all())

[(0, 0.0), (1, 1.0000000060127)]


In [ ]:
'-'*10000